<a href="https://colab.research.google.com/github/Creepyawooga/Project/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
import os

dataset_path = '/content/drive/My Drive/dataset'


In [40]:
!pip install tensorflow keras opencv-python streamlit


In [41]:
import cv2
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout

# Функция для предварительной обработки изображений с изменением размера
def preprocess_image(image_path, target_size=(128, 128)):
    if not os.path.exists(image_path):
        print(f"File {image_path} does not exist.")
        return None
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image {image_path}.")
        return None
    image = cv2.resize(image, target_size)  # Изменение размера до целевого размера
    image = image / 255.0  # Нормализация
    return image

# Функция для загрузки изображений из папки
def load_images_from_folder(folder, target_size=(128, 128)):
    images = []
    labels = []
    label = folder.split('/')[-1]  # Название папки - метка
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = preprocess_image(img_path, target_size)
        if img is not None:
            images.append(img)
            labels.append(label)
    return images, labels

# Список категорий для классификации (без bikes и planes)
categories = ['cars', 'buses']
data_dir = '/content/drive/My Drive/dataset'

all_images = []
all_labels = []

# Определяем целевой размер изображений
target_size = (128, 128)

# Загружаем изображения и их метки из папок
for category in categories:
    folder_path = os.path.join(data_dir, category)
    images, labels = load_images_from_folder(folder_path, target_size)
    all_images.extend(images)
    all_labels.extend(labels)

# Преобразование строковых меток в числовые
label_encoder = LabelEncoder()
all_labels = label_encoder.fit_transform(all_labels)

# Преобразование меток в one-hot векторы
all_labels = to_categorical(all_labels)

# Преобразование списка изображений в numpy массив
all_images = np.array(all_images)

# Печать меток для проверки
print("Labels:", all_labels)
print("Label classes:", label_encoder.classes_)

# Определяем и компилируем модель с добавлением Dropout
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=target_size + (3,)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Добавляем Dropout для регуляризации
    Dense(len(categories), activation='softmax')  # Количество классов
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Обучаем модель с увеличенным числом эпох
history = model.fit(all_images, all_labels, epochs=20, validation_split=0.2, batch_size=128)

# Сохраняем модель
model.save('vehicle_recognition_model.h5')



Labels: [[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Label classes: ['buses' 'cars']
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_42 (Conv2D)          (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_42 (MaxPooli  (None, 63, 63, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_43 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_43 (MaxPooli  (None, 30, 30, 64)        0         
 ng2D)                                                           
                              

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [42]:
%%writefile app.py
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

model = load_model('vehicle_recognition_model.h5')

categories = ['cars', 'buses', 'bikes', 'planes']

st.title("Система распознавания транспортных средств")

uploaded_file = st.file_uploader("Выберите изображение...", type="jpg")

if uploaded_file is not None:
    img = image.load_img(uploaded_file, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    predicted_class = categories[np.argmax(predictions)]

    st.image(img, caption='Загруженное изображение', use_column_width=True)
    st.write(f"Предсказанный класс: {predicted_class}")



Overwriting app.py


In [43]:
!streamlit run app.py & npx localtunnel --port 8501






  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.91.23.121:8501

npx: installed 22 in 2.905s
your url is: https://fast-singers-kneel.loca.lt
2024-07-08 07:43:43.551505: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 07:43:43.551568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 07:43:43.553515: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-08 07:43:44.967652: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find